In [1]:
%pylab inline
import os
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000, 1)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000, 1)


In [7]:
#data split : train, val, test. In addition we'll create a small development
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# The validation set will be num_validation points from the original
# training set

mask = range(num_training, num_training+num_validation)
X_val = X_train[mask]
y_val = y_train[mask]


# The training set will be num_train points from the original
# training set
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]


# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

IndexError: index 49000 is out of bounds for axis 0 with size 5000